### Run job locally and on AWS

In [17]:
from bokeh.io import output_notebook
# This tells Bokeh to display output in the notebook versus opening a browser window
output_notebook()
! pip3 list |grep bloq

Loading BokehJS ...

bloqade                           0.15.14


In [89]:
# toy problem with local detuning
from bloqade import start, save
import time

# Define relevant parameters
durations = [0.8, "sweep_time", 0.8]  # (us)
register = start.add_position([(0,0),(5,0),(10,0)])  # (um)
omega_max=15.7  # (rad/us)
delta_glob_max=124.  # (rad/us)
delta_loc_max=-1.1  # (rad/us)

# add local detuning
atomLab=[0,1,2]  # atom names labeled by their placement order. Counting from 0?
atomScale=[0.7,0.5,0.3]  # (range [0,1])  those are  modulations for the common local detune

prog = (
    register
    .rydberg.rabi.amplitude.uniform.piecewise_linear(
        durations, values=[0.0, "rabi_drive", "rabi_drive", 0.0])
    .rydberg.detuning.uniform.piecewise_linear(
        durations, values=["detune_min","detune_min","detune_max","detune_max"])
    .location(atomLab,atomScale).piecewise_linear(
        durations, values=[0.,0.,"detune_local",0.0])
)

job = prog.assign(sweep_time=1.2,rabi_drive=omega_max,
            detune_min=-delta_glob_max, detune_max=delta_glob_max, detune_local=delta_loc_max)
#print(program)
t0=time.time()
# your choice:
#emu_batch = job.braket.local_emulator().run(100)  # run with Braket-AWS
emu_batch = job.bloqade.python().run(100)   # run with Bloquade-python

print('run elaT=%.1f sec'%(time.time()-t0))
report = emu_batch.report()
counts=report.counts()   # does it remove 'e' from pre-measure and is '1'=rydberg?
print('Counts:',counts)


run elaT=0.1 sec
Counts: [OrderedDict({'010': 68, '101': 23, '110': 8, '011': 1})]


In [19]:
register.show()

In [93]:
#print(prog)

In [21]:
report.show()

0.0 10000000.0 0.0 0.0


In [29]:
hw_batch = job.braket.aquila().run_async(10,use_experimental=True)
print('run elaT=%.1f sec'%(time.time()-t0))

run elaT=8.4 sec


In [31]:
save(hw_batch,'myDetune.json')

In [87]:
import json
from pprint import pprint
from braket.aws import AwsQuantumTask

# Load the job ID from the JSON file
with open('myDetune.json', 'r') as f:
    data = json.load(f)['bloqade.task.batch.RemoteBatch']['tasks'][0][1]['bloqade.task.braket.BraketTask']
    #pprint(data)
    job_id = data['task_id']
    #pprint(data['bloqade.task.batch.RemoteBatch'])
#1job_id='arn:aws:braket:us-east-1:765483381942:quantum-task/ee191b55-cfa9-4b11-a1ba-a578c3567a06'
print(' Retrieve the job',job_id)
hw_batch3 = AwsQuantumTask(arn=job_id)

 Retrieve the job arn:aws:braket:us-east-1:765483381942:quantum-task/ee191b55-cfa9-4b11-a1ba-a578c3567a06


In [88]:
# Check the status of the job
status = hw_batch3.state()
print(f"Job status: {status}")
# Get the results if the job is completed
if status == 'COMPLETED':
    result = hw_batch3.result()
    #print(result)
    #print(dir(result))
    counts = result.get_counts()
    #counts=result.counts()   # does it remove 'e' from pre-measure and is '1'=rydberg?
    print('Counts3:',counts)


Job status: COMPLETED
Counts3: {'rgr': 9, 'ggr': 1}


In [82]:
result.task_metadata

TaskMetadata(braketSchemaHeader=BraketSchemaHeader(name='braket.task_result.task_metadata', version='1'), id='arn:aws:braket:us-east-1:765483381942:quantum-task/ee191b55-cfa9-4b11-a1ba-a578c3567a06', shots=10, deviceId='arn:aws:braket:us-east-1::device/qpu/quera/Aquila', deviceParameters=None, createdAt='2024-11-16T03:31:19.515Z', endedAt='2024-11-16T03:51:06.138Z', status='COMPLETED', failureReason=None)

In [83]:
result.additional_metadata

AdditionalMetadata(action=Program(braketSchemaHeader=BraketSchemaHeader(name='braket.ir.ahs.program', version='1'), setup=Setup(ahs_register=AtomArrangement(sites=[[Decimal('0.0'), Decimal('0.0')], [Decimal('0.000005'), Decimal('0.0')], [Decimal('0.00001'), Decimal('0.0')]], filling=[1, 1, 1])), hamiltonian=Hamiltonian(drivingFields=[DrivingField(amplitude=PhysicalField(time_series=TimeSeries(values=[Decimal('0.0'), Decimal('15700000.0'), Decimal('15700000.0'), Decimal('0.0')], times=[Decimal('0.0'), Decimal('8E-7'), Decimal('0.000002'), Decimal('0.0000028')]), pattern='uniform'), phase=PhysicalField(time_series=TimeSeries(values=[Decimal('0.0'), Decimal('0.0')], times=[Decimal('0.0'), Decimal('0.0000028')]), pattern='uniform'), detuning=PhysicalField(time_series=TimeSeries(values=[Decimal('-124000000.0'), Decimal('-124000000.0'), Decimal('124000000.0'), Decimal('124000000.0')], times=[Decimal('0.0'), Decimal('8E-7'), Decimal('0.000002'), Decimal('0.0000028')]), pattern='uniform'))], l